In [ ]:
from neo4j import GraphDatabase
import json
from Modules.process_pubmed_data import *

In [ ]:
# Network Analysis in Neo4J

# Medical Devices
# Create author data file
with open("JSON Data/device_pubmed_data.json", "r") as f:
    pubmed_data = json.load(f)
get_author_info(pubmed_data=pubmed_data, write_file="JSON Data/device_author_data.json")

# Extract and clean affiliations
with open("JSON Data/device_author_data.json", "r") as f:
    data = json.load(f)
match_affiliation(data, 90, "JSON Data/device_institution_match.json")

# Biomedical
with open("JSON Data/biomed_pubmed_data.json", "r") as f:
    pubmed_data = json.load(f)
get_author_info(pubmed_data=pubmed_data, cutoff_year=2000, write_file="JSON Data/biomed_author_data.json")

with open("JSON Data/biomed_author_data.json", "r") as f:
    data = json.load(f)
match_affiliation(data, 90, "JSON Data/biomed_institution_match.json")

## Neo4J
# Pick the correct driver for target local database
#driver = GraphDatabase.driver("neo4j://localhost:7687", auth=("neo4j", "devices"))
driver = GraphDatabase.driver("neo4j://localhost:7687", auth=("neo4j", "biomed"))
def create_author_node(tx, author_name, author_data):
        tx.run("MERGE (a: author {id: $id, name: $name, years_active: $years_active})", 
        id=author_data[author_name]["id"], name=author_name, 
        # Some authors have multiple papers published in the same year. Ex: Munzner RF
        years_active=list(set([eval(date).year for date in author_data[author_name]["years active"]])))
def add_relationships(tx, author_name, author_data):
    for coauthor in author_data[author_name]["edges"]:
        tx.run("""MATCH (a: author {id: $a_id, name: $a_name})
        MATCH (b: author {id: $b_id, name: $b_name})
        MERGE (a)-[c:COAUTHOR {date: $date}]->(b)""",
        a_id=author_data[author_name]["id"],
        a_name=author_name,
        b_id=author_data[coauthor[0]]["id"],
        b_name=coauthor[0],
        date=eval(coauthor[1]).isoformat())
